In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import sys
import os 

sys.path.append('..')
from src import config

if sys.platform == 'linux':
    path = config.LINUX_PATH
else:
    path = config.OS_PATH
os.chdir(path)

In [2]:
data_path = config.STRAVA_TRAIN_PATH
data = pd.read_csv(data_path, index_col=0)
print(data.columns)
data.head()

Index(['name', 'distance', 'moving_time', 'total_elevation_gain',
       'workout_type', 'timezone', 'achievement_count', 'kudos_count',
       'athlete_count', 'photo_count', 'manual', 'max_speed',
       'average_heartrate', 'max_heartrate', 'pr_count', 'total_photo_count',
       'suffer_score', 'date', 'time', 'run_area', 'latlng_cluster', 'city',
       'average_speed_mpk'],
      dtype='object')


name  distance  moving_time  total_elevation_gain  \
260             Pre meet     7.1960    28.383333                   3.0   
261                    WU    4.8448    22.183333                  44.0   
262        Gain train 🚂😈🤪    8.1354    36.100000                  10.0   
263  Not 10, sorry Logan    12.9277    59.283333                  49.0   
264                     😴    6.7834    30.283333                   9.0   

     workout_type       timezone  achievement_count  kudos_count  \
260           3.0  America/Boise                0.0         33.0   
261           0.0  America/Boise                0.0         11.0   
262           0.0  America/Boise                0.0         32.0   
263           0.0  America/Boise                2.0         44.0   
264           0.0  America/Boise                0.0         27.0   

     athlete_count  photo_count  ...  max_heartrate  pr_count  \
260            1.0          0.0  ...          173.0       0.0   
261            1.0          0.0  ...          167.0       0.0   
262            1.0          0.0  ...          164.0       0.0   
263            1.0          0.0  ...          165.0       0.0   
264            1.0          0.0  ...          165.0       0.0   

     total_photo_count  suffer_score        date      time   run_area  \
260                0.0          16.0  2021-01-29  21:51:05   2.207397   
261                0.0          20.0  2021-01-29  21:08:46   0.166980   
262                0.0          26.0  2021-01-29  16:01:32   5.870196   
263                0.0          52.0  2021-01-28  22:39:54  37.574116   
264                0.0          29.0  2021-01-28  14:13:57   4.207750   

    latlng_cluster   city  average_speed_mpk  
260              1  Boise           3.944615  
261              1  Boise           4.578571  
262              1  Boise           4.437167  
263              1  Boise           4.586131  
264              1  Boise           4.464506  

[5 rows x 23 columns]

In [7]:
from pycaret.regression import *

data_path = config.STRAVA_TRAIN_PATH
data = pd.read_csv(data_path, index_col=0)
data = data.drop(['name'], axis=1)
reg = setup(data, target='kudos_count')
compare_models()

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=6505, verbose=0, warm_start=False)

In [11]:
model = create_model('xgboost')
tuned_model = tune_model(model)
model_final = finalize_model(tuned_model)
# save_model(model_final, 'models/kudos_second')

IntProgress(value=0, description='Processing: ', max=7)

Initiated  . . . . . . . . . . . . . . . . . .                   20:03:20
Status     . . . . . . . . . . . . . . . . . .  Searching Hyperparameters
Estimator  . . . . . . . . . . . . . . . . . .  Extreme Gradient Boosting

Empty DataFrame
Columns: [MAE, MSE, RMSE, R2, RMSLE, MAPE]
Index: []

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 40.2min


KeyboardInterrupt: 

In [9]:
plot_model(tuned_model)
interpret_model(tuned_model)
plot_model(tuned_model, plot='feature')

In [10]:
data_path = config.STRAVA_TEST_PATH
data_unseen = pd.read_csv(data_path, index_col=0)
predictions = predict_model(tuned_model, data=data_unseen)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(predictions.kudos_count.values, predictions.Label.values)
rmse = mean_squared_error(predictions.kudos_count.values, predictions.Label.values, squared=False)
print(f'MSE: {mse:.2f}\nRMSE: {rmse:.2f}')

MSE: 270.90
RMSE: 16.46
